In [28]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../kaggle_prediction_library/') 
import preprocess
import feature_engineering
import submission
import validation
from sklearn.model_selection import train_test_split

# from hyperopt import tpe, fmin, Trials
# import hyperopt.hp as hp

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss
from sklearn.pipeline import make_pipeline



In [29]:
to_predict_mens = pd.read_csv("to_predict_mens.csv")


In [42]:
to_predict_mens_first_round_train = to_predict_mens[(to_predict_mens["GameRound"] == 1)
                                                    & (to_predict_mens.final_odds.notnull())
                                                    & (to_predict_mens.type == "Historical")
                                                    ].copy()

In [43]:
to_predict_mens_train = to_predict_mens[(to_predict_mens.type == "Historical")].copy()

In [44]:
to_predict_mens_first_round_test = to_predict_mens[(to_predict_mens.type == "Prediction")
                                                   & (to_predict_mens.GameRound == 1)
                                                   ].copy()

In [45]:
to_predict_mens_other_rounds_test = to_predict_mens[(to_predict_mens.type == "Prediction")
                                                   & (to_predict_mens.GameRound > 1)
                                                   ].copy()


### Odds Model Training

In [46]:
best_params = {"C": .1}
model = LogisticRegression(**best_params)
pipeline = make_pipeline(StandardScaler(), model)
odds_model = pipeline.fit(to_predict_mens_first_round_train[["final_odds"]], to_predict_mens_first_round_train["Outcome"])


### Statistics Model Training

In [47]:
best_params = {"C": .1}
model = LogisticRegression(**best_params)
pipeline = make_pipeline(StandardScaler(), model)
features = ['t1_adj_margin', 't2_adj_margin', 't1_final_rank', 't2_final_rank', 't1_OrdinalRank', 't2_OrdinalRank']
statistics_model = pipeline.fit(to_predict_mens_train[features], to_predict_mens_train["Outcome"])


### Inference

In [56]:
pred_proba = odds_model.predict_proba(to_predict_mens_first_round_test[["final_odds"]].copy())[:,1]
to_predict_mens_first_round_test["odds_pred"] = pred_proba


In [54]:
pred_proba = statistics_model.predict_proba(to_predict_mens_first_round_test[features].copy())[:,1]
to_predict_mens_first_round_test["statistics_pred"] = pred_proba


In [58]:
to_predict_mens_first_round_test["Pred"] = (to_predict_mens_first_round_test.odds_pred * .75) + \
                                            (to_predict_mens_first_round_test.statistics_pred * .25)


In [61]:
pred_proba = statistics_model.predict_proba(to_predict_mens_other_rounds_test[features].copy())[:,1]
to_predict_mens_other_rounds_test["Pred"] = pred_proba


In [68]:
mens_sub = pd.concat([to_predict_mens_first_round_test[["ID", "Pred"]],
           to_predict_mens_other_rounds_test[["ID", "Pred"]]], axis=0)

In [72]:
mens_sub.to_csv("../submissions/sub_2023_in_2024.csv")